# Syed Muhammad Hamza  (MSAI-F22-M010)
# Task Assigned is VIVA as Project Improvements
#### => Audio record and play button
#### => Realtime audio recording and prediction 
#### => True and predicted label for Activity Monitoring 

# Please run all testing cells (don't run training cells all models are trained already)

# Activity Monitoring

## Training

In [1]:
import numpy as np
from scipy import signal
from scipy.stats import kurtosis, skew
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import joblib
import time

#For Loading Data
def load_data(data_set):
        # Load or define the labels

        file_names = {
            'training': [
                'trainLabels.npy',
                'trainAccelerometer.npy',
                'trainGravity.npy',
                'trainGyroscope.npy',
                'trainJinsAccelerometer.npy',
                'trainJinsGyroscope.npy',
                'trainLinearAcceleration.npy',
                'trainMagnetometer.npy',
                'trainMSAccelerometer.npy',
                'trainMSGyroscope.npy'
            ],
            'testing': [
                'testLabels.npy',
                'testAccelerometer.npy',
                'testGravity.npy',
                'testGyroscope.npy',
                'testJinsAccelerometer.npy',
                'testJinsGyroscope.npy',
                'testLinearAcceleration.npy',
                'testMagnetometer.npy',
                'testMSAccelerometer.npy',
                'testMSGyroscope.npy'
            ]
        }
        data = []
        for file_name in file_names[data_set]:
            file_path = data_dir + data_set + '/' + file_name
            loaded_data = np.load(file_path)
            data.append(loaded_data)
            print(loaded_data.shape, "\t", file_name)
        return data

In [49]:
#Main Class for training and testing
class ActivityMonitor:
    def __init__(self, train_data, test_data, train_labels, test_labels):
        self.train_data = train_data
        self.test_data = test_data
        self.train_labels = train_labels
        self.test_labels = test_labels
        self.path_save_model = "ActivityMonitoring/TrainedModels/"
        
    def normalize_data(self, data):
        reshaped_data = np.reshape(data, (-1, data.shape[2]))
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)
        normalized_data = (data - mean) / std
        normalized_data = np.reshape(normalized_data, data.shape)
        return normalized_data

    def resample_data(self, data, target_frequency):
        current_frequency = data.shape[1] / 4.0
        resampled_data = signal.resample(data, int(data.shape[1] * target_frequency / current_frequency), axis=1)
        return resampled_data

    def extract_features(self, data):
        mean = np.mean(data, axis=1)
        std = np.std(data, axis=1)
        min_val = np.min(data, axis=1)
        max_val = np.max(data, axis=1)
        median = np.median(data, axis=1)
        kurt = kurtosis(data, axis=1)
        skewness = skew(data, axis=1)
        features = np.concatenate([mean, std, min_val, max_val, median, kurt, skewness], axis=1)
        return features

    def process_data(self, data):
        sensor_frequencies = {
            1: 200,  # Accelerometer
            2: 200,  # Gravity
            3: 200,  # Gyroscope
            4: 20,   # JinsAccelerometer
            5: 20,   # JinsGyroscope
            6: 200,  # LinearAcceleration
            7: 50,   # Magnetometer
            8: 67,   # MSAccelerometer
            9: 67    # MSGyroscope
        }
        processed_data = []
        for i, d in enumerate(data[1:], start=1):
            if i in sensor_frequencies:
                target_frequency = 200
                resampled_data = self.resample_data(d, sensor_frequencies[i])
                normalized_data = self.normalize_data(resampled_data)
                extracted_features = self.extract_features(normalized_data)
                processed_data.append(extracted_features)
            else:
                processed_data.append(d)
        processed_data.insert(0, data[0])
        return processed_data

    def preprocess_data(self):
        normalized_train = self.process_data(self.train_data)
        normalized_test = self.process_data(self.test_data)
        train_labels = normalized_train[0]
        self.train_data = np.concatenate(normalized_train[1:], axis=1).reshape(len(train_labels), -1)
        test_labels = normalized_test[0]
        self.test_data = np.concatenate(normalized_test[1:], axis=1).reshape(len(test_labels), -1)
#------------------------------------------------------------------------------------------------------------------------------------------
    def calculate_evaluation_metrics(self, true_labels, predicted_labels, set_name):
        acc = accuracy_score(true_labels, predicted_labels)
        cm = confusion_matrix(true_labels, predicted_labels)
        f1 = f1_score(true_labels, predicted_labels, average='weighted')
        precision = precision_score(true_labels, predicted_labels, average='weighted', zero_division=1)
        recall = recall_score(true_labels, predicted_labels, average='weighted')
        print("\n")
        print(set_name, "Accuracy:", acc)
        print(set_name, "Precision:", precision)
        print(set_name, "F1 Score:", f1)
        print(set_name, "Recall:", recall)
        print(set_name, "Confusion Matrix:\n", cm)
        return acc, cm, f1, precision, recall
#------------------------------------------------------------------------------------------------------------------------------------------ 
    #Random Forest Classifier
    def random_forest_classifier(self):
        classifier = RandomForestClassifier()
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val)
        joblib.dump(classifier,self.path_save_model + "random_forest_classifier.joblib")
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
        
    #Finding Hyperparameters Random Forest Grid
    # def random_forest_grid_search(self):
    #     param_grid = {
    #         'n_estimators': [100, 200, 300],
    #         'max_depth': [None, 5, 10],
    #         'min_samples_split': [2, 5, 10]
    #     }
    #     classifier = RandomForestClassifier()
    #     grid_search = GridSearchCV(classifier, param_grid, cv=5)
    #     grid_search.fit(self.train_data, self.train_labels)
    #     best_classifier = grid_search.best_estimator_
    #     print("Selected Hyperparameters:", best_classifier.get_params(), "\n\n")
    #     cv_scores = cross_val_score(best_classifier, self.train_data, self.train_labels, cv=5)
    #     print("Cross-validation scores:", cv_scores)
    #     print("Mean CV score:", np.mean(cv_scores))
    #     X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
    #     best_classifier.fit(self.train_data, self.train_labels)
    #     val_predictions = best_classifier.predict(self.X_val)
    #     joblib.dump(best_classifier,self.path_save_model + "random_forest_grid_search.joblib")
    #     val_metrics = self.calculate_evaluation_metrics(self.y_val, val_predictions, "Validation")
    #     test_predictions = best_classifier.predict(self.test_data)
    #     test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
    #     return best_classifier
        
    #Random Forest Grid with trained hyperparameters
    def random_forest_grid_search_hyperparameters(self):
        classifier = RandomForestClassifier(
            bootstrap=True,
            ccp_alpha=0.0,
            class_weight=None,
            criterion='gini',
            max_depth=None,
            # max_features='auto',
            max_leaf_nodes=None,
            max_samples=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=1,
            min_samples_split=2,
            min_weight_fraction_leaf=0.0,
            n_estimators=300,
            n_jobs=None,
            oob_score=False,
            random_state=None,
            verbose=0,
            warm_start=False
        )
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        # Fit the classifier on the training data
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val) 
        # Save model
        joblib.dump(classifier,self.path_save_model + "random_forest_grid_search_hyperparameters.joblib")
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
#------------------------------------------------------------------------------------------------------------------------------------------
    #SVM Classifier
    def svm_classifier(self):
        classifier = SVC()
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val)
        joblib.dump(classifier,self.path_save_model + "svm_classifier.joblib")
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
    
    #Finding Hyperparameters SVM Grid
    # def svm_grid_search(self):
    #     param_grid = {
    #         'C': [0.1, 1, 10],
    #         'kernel': ['linear', 'rbf', 'sigmoid'],
    #         'gamma': [0.1, 1, 'scale']
    #     }
    #     classifier = SVC()
    #     grid_search = GridSearchCV(classifier, param_grid, cv=5)
    #     X_train, X_val, y_train, y_val = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)
    # 
    #     grid_search.fit(self.train_data, self.train_labels)
    #     best_classifier = grid_search.best_estimator_
    #     print("Selected Hyperparameters:", best_classifier.get_params(), "\n\n")
    #     cv_scores = cross_val_score(best_classifier, self.train_data, self.train_labels, cv=5)
    #     print("Cross-validation scores:", cv_scores)
    #     print("Mean CV score:", np.mean(cv_scores))
    #     best_classifier.fit(self.train_data, self.train_labels)
    #     val_predictions = best_classifier.predict(self.X_val)
    #     joblib.dump(best_classifier,"ActivityMonitoring/svm_grid_search.joblib")
    #     val_metrics = self.calculate_evaluation_metrics(self.y_val, val_predictions, "Validation")
    #     test_predictions = best_classifier.predict(self.test_data)
    #     test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
    #     return best_classifier
    
    #SVM Grid with trained hyperparameters
    def svm_grid_search_hyperparameters(self):
        classifier = SVC(
            C=0.1,
            break_ties=False,
            cache_size=200,
            class_weight=None,
            coef0=0.0,
            decision_function_shape='ovr',
            degree=3,
            gamma=0.1,
            kernel='linear',
            max_iter=-1,
            probability=False,
            random_state=None,
            shrinking=True,
            tol=0.001,
            verbose=False,
        )
        X_train, X_val, y_train, y_val = train_test_split(self.train_data, self.train_labels, test_size=0.2, random_state=42)
        # Fit the classifier on the training data
        classifier.fit(X_train, y_train)
        val_predictions = classifier.predict(X_val) 
        # Save model
        joblib.dump(classifier,self.path_save_model + "svm_grid_search_hyperparameters.joblib")
        cv_scores = cross_val_score(classifier, self.train_data, self.train_labels, cv=5)
        print("Cross-validation scores:", cv_scores)
        print("Mean CV score:", np.mean(cv_scores))
        val_metrics = self.calculate_evaluation_metrics(y_val, val_predictions, "Validation")
        test_predictions = classifier.predict(self.test_data)
        test_metrics = self.calculate_evaluation_metrics(self.test_labels, test_predictions, "Test")
        return classifier
        
# Example usage:
if __name__ == "__main__":

    data_dir = 'ActivityMonitoring/'
    train_data = load_data("training")
    test_data = load_data("testing")
    train_labels = np.load(data_dir + 'training/trainLabels.npy')
    test_labels = np.load(data_dir + 'testing/testLabels.npy')

    data_loader = ActivityMonitor(train_data, test_data, train_labels, test_labels)
    data_loader.preprocess_data()

    # Random Forest Classifier
    random_forest_clf = data_loader.random_forest_classifier()
    random_forest_clf_grid = data_loader.random_forest_grid_search_hyperparameters()

    # SVM Classifier
    svm_clf = data_loader.svm_classifier()
    svm_grid_clf = data_loader.svm_grid_search_hyperparameters()
    
    
    

(2284,) 	 trainLabels.npy
(2284, 800, 3) 	 trainAccelerometer.npy
(2284, 800, 3) 	 trainGravity.npy
(2284, 800, 3) 	 trainGyroscope.npy
(2284, 80, 3) 	 trainJinsAccelerometer.npy
(2284, 80, 3) 	 trainJinsGyroscope.npy
(2284, 800, 3) 	 trainLinearAcceleration.npy
(2284, 200, 3) 	 trainMagnetometer.npy
(2284, 268, 3) 	 trainMSAccelerometer.npy
(2284, 268, 3) 	 trainMSGyroscope.npy
(2288,) 	 testLabels.npy
(2288, 800, 3) 	 testAccelerometer.npy
(2288, 800, 3) 	 testGravity.npy
(2288, 800, 3) 	 testGyroscope.npy
(2288, 80, 3) 	 testJinsAccelerometer.npy
(2288, 80, 3) 	 testJinsGyroscope.npy
(2288, 800, 3) 	 testLinearAcceleration.npy
(2288, 200, 3) 	 testMagnetometer.npy
(2288, 268, 3) 	 testMSAccelerometer.npy
(2288, 268, 3) 	 testMSGyroscope.npy
Cross-validation scores: [0.74835886 0.75492341 0.78118162 0.75273523 0.75657895]
Mean CV score: 0.7587556144189798


Validation Accuracy: 0.7592997811816192
Validation Precision: 0.7895745795146925
Validation F1 Score: 0.7562808340377086
Validat

## Testing

In [1]:
import numpy as np
import joblib

def predict_activity(inp, model_no):
    path_save_model = "ActivityMonitoring/TrainedModels/"
    activity_labels = {
        0: "Bring",
        1: "Clean Floor",
        2: "Clean Surface",
        3: "Close Big Box",
        4: "Close Door",
        5: "Close Drawer",
        6: "Close Lid By Rotate",
        7: "Close Other Lid",
        8: "Close Small Box",
        9: "Close Tap Water",
        10: "Drink",
        11: "Dry Off Hands",
        12: "Dry Off Hands By Shake",
        13: "Eat Small",
        14: "Gargle",
        15: "Getting Up",
        16: "Hang",
        17: "Lying Down",
        18: "Open Bag",
        19: "Open Big Box",
        20: "Open Door",
        21: "Open Drawer",
        22: "Open Lid By Rotate",
        23: "Open Other Lid",
        24: "Open Small Box",
        25: "Open Tap Water",
        26: "Plug In",
        27: "Press By Grasp",
        28: "Press From Top",
        29: "Press Switch",
        30: "Put From Bottle",
        31: "Put From Tap Water",
        32: "Put High Position",
        33: "Put On Floor",
        34: "Read",
        35: "Rotate",
        36: "Rub Hands",
        37: "Scoop And Put",
        38: "Sitting Down",
        39: "Squatting Down",
        40: "Standing Up",
        41: "Stand Up From Squatting",
        42: "Take From Floor",
        43: "Take From High Position",
        44: "Take Off Jacket",
        45: "Take Out",
        46: "Talk By Telephone",
        47: "Throw Out",
        48: "Throw Out Water",
        49: "Touch Smartphone Screen",
        50: "Type",
        51: "Unhang",
        52: "Unplug",
        53: "Wear Jacket",
        54: "Write"
    }
    
    classifier = joblib.load(path_save_model + "random_forest_classifier.joblib")
    
    if(model_no == "Random Forest"):
        classifier = joblib.load(path_save_model + "random_forest_classifier.joblib")    
    elif(model_no == "Random Forest Grid"):
        classifier = joblib.load(path_save_model + "random_forest_grid_search_hyperparameters.joblib")
    elif(model_no == "SVM"):
        classifier = joblib.load(path_save_model + "svm_classifier.joblib")
    elif(model_no == "SVM Grid"):
        classifier = joblib.load(path_save_model + "svm_grid_search_hyperparameters.joblib")
    
    #arr = [data_loader.test_data[int(inp), :]]
    true_label = inp[0]
    arr = inp[1:]

    np_arr = np.array(arr)
    #print(np_arr.shape)
    
    prediction = classifier.predict(np_arr.reshape(1, -1))
    
    #print(prediction[0])
        
    activity = activity_labels.get(prediction[0])
    
    print(activity)
    true_label_concat = "Label = " + str(true_label) + "\tActivity = " + activity_labels.get(true_label)
    predicted_label_concat = "Label = " + str(prediction[0]) + "\tActivity = " + activity
    
    return true_label_concat, predicted_label_concat

In [2]:
import gradio as gr

def predict_wrapper(csv_file_path, model):
    data = np.genfromtxt(csv_file_path, delimiter=',')    
    return predict_activity(data, model)

activity_prediction = gr.Interface(
    fn=predict_wrapper,  # Pass your function
    inputs=[
        gr.File(label="Upload CSV File"),
        #gr.Textbox(type="text", label="Model No", placeholder="Enter model to test")
        gr.Radio(choices=["Random Forest", "Random Forest Grid", "SVM", "SVM Grid"], value="Random Forest")
    ],
    outputs=[
        gr.Textbox(type="text", label="True Label"),
        gr.Textbox(type="text", label="Prediction")
    ]  # Output component
)
activity_prediction.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


# Gender Recognition

# Training

In [3]:
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
import pickle

def load_data(vector_length=128):
    """A function to load gender recognition dataset from `data` folder
    After the second run, this will load from results/features.npy and results/labels.npy files
    as it is much faster!"""
    # make sure results folder exists
    #if not os.path.isdir("results"):
    #    os.mkdir("results")
    # if features & labels already loaded individually and bundled, load them from there instead
    if os.path.isfile("GenderRecognition/features.npy") and os.path.isfile("GenderRecognition/labels.npy"):
        X = np.load("GenderRecognition/features.npy")
        y = np.load("GenderRecognition/labels.npy")
        return X, y
    
X, y = load_data()

def split_data(X, y, test_size=0.1, valid_size=0.1):
    # split training set and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=7)
    # split training set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }

# load the dataset
X, y = load_data()
# split the data into training, validation and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)

def create_model(vector_length=128):
    """5 hidden dense layers from 256 units to 64, not the best model."""
    model = Sequential()
    model.add(Dense(256, input_shape=(vector_length,)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    # one output neuron with sigmoid activation function, 0 means female, 1 means male
    model.add(Dense(1, activation="sigmoid"))
    # using binary crossentropy as it's male/female classification (binary)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    # print summary of the model
    print(model.summary())
    return model

# construct the model
model = create_model()

# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100
# train the model using the training set and validating using validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# save the model to a file
#model.save("results/model.h5")
pickle.dump(model, open("GenderRecognition/TrainedModels/gender.pickle", 'wb'))


# evaluating the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")#6694
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
#0.2143
print(f"Accuracy: {accuracy*100:.2f}%")
#92.02%

def conf_matrix():
    # Create the confusion matrix values
    cm = confusion_matrix(data["y_test"][:10000], svm_predictions)

    # Create the confusion matrix display
    plt.figure(figsize=(8,8))
    plt.title('Confusion matrix on test data')
    sns.heatmap(cm, annot=True, fmt='d', 
                cmap=plt.cm.Blues, cbar=False, annot_kws={'size':14})
    #to visualise a confusion matrix, time-series movements, temperature changes, correlation matrix and SHAP interaction values
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

NameError: name 'os' is not defined

# Testing

In [3]:
import argparse
import librosa
import pickle
import os
"""
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
def extract_feature(file_name, **kwargs):
    
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X)) #analyze the frequency content of a signal over time
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
args = parser.parse_args()


In [4]:
def test_sample(file):
    path_to_model = "GenderRecognition/TrainedModels/"
    # if not file or not os.path.isfile(file):
    #     return "file doesn't exist"
    # else:
    loaded_model = pickle.load(open(path_to_model+"gender.pickle", 'rb'))
    features = extract_feature(file, mel=True).reshape(1, -1)
    # predict the gender!
    print(loaded_model.predict(features))
    male_prob = loaded_model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    #print("Result:", gender)
    #print(f"Probabilities::: Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")
    result = "Result: " + gender + ", Probabilities::: Male: " + f"{male_prob*100:.2f}" + "   Female: " + f"{female_prob*100:.2f}"
    print(result)
    return result

In [5]:
import gradio as gr
def gender_recoginition(test_example):
    return test_sample(test_example)

gender_prediction_with_file = gr.Interface(
    fn = gender_recoginition,    
    inputs = [
        gr.Audio(sources=["upload"], show_download_button=True, format="wav", type="filepath")
        #gr.File(label="Upload audio file...")
    ],
    outputs = "text",
    live=True,
)
gender_prediction_with_file.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


# Hunger Detection

## Testing

In [19]:
from sklearn.impute import SimpleImputer
import pandas as pd
import joblib

def DetectHunger(main_dir, data, model):
    # data = pd.read_csv(file)    

    X = data.drop('Label', axis=1)
    y = data['Label']

    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)

    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    if(model == "RF_15"):
        model = joblib.load(f"{main_dir}TrainedModels/randomforest_15_interval_model.joblib")
    elif(model == "RF_30"):
        model = joblib.load(f"{main_dir}TrainedModels/randomforest_30_interval_model.joblib")
    elif(model == "RF_60"):
        model = joblib.load(f"{main_dir}TrainedModels/randomforest_60_interval_model.joblib")
        
    elif(model == "SVM_15"):
        model = joblib.load(f"{main_dir}TrainedModels/svm_15_interval_model.joblib")
    elif(model == "SVM_30"):
        model = joblib.load(f"{main_dir}TrainedModels/svm_30_interval_model.joblib")
    elif(model == "SVM_60"):
        model = joblib.load(f"{main_dir}TrainedModels/svm_60_interval_model.joblib")
        
    elif(model == "NB_15"):
        model = joblib.load(f"{main_dir}TrainedModels/naivebayes_15_interval_model.joblib")
    elif(model == "NB_30"):
        model = joblib.load(f"{main_dir}TrainedModels/naivebayes_30_interval_model.joblib")
    elif(model == "NB_60"):
        model = joblib.load(f"{main_dir}TrainedModels/naivebayes_60_interval_model.joblib")
        
    elif(model == "LR_15"):
        model = joblib.load(f"{main_dir}TrainedModels/logisticregression_15_interval_model.joblib")
    elif(model == "LR_30"):
        model = joblib.load(f"{main_dir}TrainedModels/logisticregression_30_interval_model.joblib")
    elif(model == "LR_60"):
        model = joblib.load(f"{main_dir}TrainedModels/logisticregression_60_interval_model.joblib")
        
    prediction = model.predict(X)
    pred = ""
    
    if(y.iloc[0] == 0):
        pred = "Not Hungry"
    elif(y.iloc[0] == 1):
        pred = "Hungry"
    
    true_label = f"Label={y.iloc[0]}\tPrediction={pred}"
       
    if(prediction[0] == 0):
        pred = "Not Hungry"
    elif(prediction[0] == 1):
        pred = "Hungry"
    
    predicted = f"Label={prediction[0]}\tPrediction={pred}" 
    
    return true_label, predicted



In [22]:
main_dir = "HungerDetection/"
file_path = f"{main_dir}TestExamples/t4.csv"
file = pd.read_csv(file_path)
DetectHunger(main_dir, file, "NB_60")
#type(true_label)
#type(prediction[0])

('Label=0\tPrediction=Not Hungry', 'Label=1\tPrediction=Hungry')

# Combined 

In [23]:
import numpy as np
import gradio as gr

def predict_wrapper(csv_file_path, model):
    data = np.genfromtxt(csv_file_path, delimiter=',')
    return predict_activity(data, model)

def gender_recoginition(test_example):
    return test_sample(test_example)

def detect_Wrapper(csv_file_path, model):
    data = pd.read_csv(csv_file_path)
    main_dir = "HungerDetection/"
    return DetectHunger(main_dir, data, model)

activity_prediction = gr.Interface(
    fn=predict_wrapper,  # Pass your function
    inputs=[
        gr.File(label="Upload CSV File"),
        #gr.Textbox(type="text", label="Model No", placeholder="Enter model to test")
        gr.Radio(choices=["Random Forest", "Random Forest Grid", "SVM", "SVM Grid"], value="Random Forest")
    ],
    outputs=[
        gr.Textbox(type="text", label="True Label"),
        gr.Textbox(type="text", label="Prediction")
    ]  # Output component
)

hunger_detection = gr.Interface(
    fn=detect_Wrapper,  # Pass your function
    inputs=[
        gr.File(label="Upload CSV File"),
        #gr.Textbox(type="text", label="Model No", placeholder="Enter model to test")
        gr.Radio(choices=[
            "RF_15", "RF_30", "RF_60", 
            "SVM_15", "SVM_30", "SVM_60", 
            "LR_15", "LR_30", "LR_60",
            "NB_15", "NB_30", "NB_60"
        ], value="RF_15")
    ],
    outputs=[
        gr.Textbox(type="text", label="True Label"),
        gr.Textbox(type="text", label="Prediction")
    ]  # Output component
)

gender_prediction_with_file = gr.Interface(
    fn = gender_recoginition,    
    inputs = [
        gr.Audio(sources=["upload"], show_download_button=True, format="wav", type="filepath")
        #gr.File(label="Upload audio file...")
    ],
    outputs = "text",
    live=True,
)


gender_prediction_realtime = gr.Interface(
    fn = gender_recoginition,    
    inputs = [
        gr.Audio(sources=["microphone"], show_download_button=True, format="wav", type="filepath")
    ],
    outputs = "text",
    live=True,
)

group = gr.TabbedInterface([activity_prediction,hunger_detection,gender_prediction_with_file,gender_prediction_realtime],tab_names=["Activity Recognition", "Hunger Detection", "Gender Classification With File", "Gender Classification Realtime"])

# Launch the group interface
group.launch(share=True)



D:\UniversityData\PycharmProjects\venv\lib\site-packages\gradio\blocks.py:946: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(
D:\UniversityData\PycharmProjects\venv\lib\site-packages\gradio\blocks.py:946: UserWarning: api_name predict already exists, using predict_2
  warnings.warn(
D:\UniversityData\PycharmProjects\venv\lib\site-packages\gradio\blocks.py:946: UserWarning: api_name predict already exists, using predict_3
  warnings.warn(


Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://9140ea2fb86826a223.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
